In [48]:
import os
import requests
from tqdm import tqdm
import gzip
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import warnings 

output_dir = "data/srtm1_hgt_tiles"
os.makedirs(output_dir, exist_ok=True)

def format_tile_name(lat, lon):
    lat_prefix = 'N' if lat >= 0 else 'S'
    lon_prefix = 'E' if lon >= 0 else 'W'
    return f"{lat_prefix}{abs(lat):02d}{lon_prefix}{abs(lon):03d}"

def generate_us_tile_names():
    tiles = []
    for lat in range(24, 61):      # 24N to 49N
        for lon in range(-125, -65):  # 125W to 66W
            tile_name = format_tile_name(lat, lon)
            tiles.append(tile_name)
    return tiles

def generate_alaska_tile_names():
    tiles = []
    for lat in range(51, 73):      # 51N to 72N
        for lon in range(-180, -130):  # 180W to 130W
            tile_name = format_tile_name(lat, lon)
            tiles.append(tile_name)
    return tiles

def build_srtm1_urls(tile_names):
    urls = []
    for name in tile_names:
        folder = name[:3] 
        url = f"https://s3.amazonaws.com/elevation-tiles-prod/skadi/{folder}/{name}.hgt.gz"
        urls.append((name, url))
    return urls

def download_tiles(urls, output_dir):
    for name, url in tqdm(urls, desc="Downloading tiles"):
        file_path = os.path.join(output_dir, f"{name}.hgt.gz")
        if os.path.exists(file_path):
            continue  # Skip already downloaded
        try:
            response = requests.get(url, stream=True, timeout=30)
            if response.status_code == 200:
                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(8192):
                        f.write(chunk)
            else:
                print(f"Failed: {url} {response.status_code}")
        except Exception as e:
            print(f"Error downloading {url}: {e}")

tile_names = generate_us_tile_names()
tile_urls = build_srtm1_urls(tile_names)

#missing_tiles = ['S89E100', 'S89E100', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W145', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W145', 'N60W149', 'N60W149', 'N60W149', 'N60W148', 'N60W149', 'N59W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N58W135', 'N55W132', 'N56W132', 'N57W135', 'N55W132', 'N57W135', 'N57W134', 'N55W131', 'N55W133', 'N58W134', 'N55W133', 'N55W132', 'N58W134', 'N58W134', 'N56W132', 'N58W135', 'N58W135', 'N58W134', 'N56W132', 'N58W134', 'N58W134', 'N58W134', 'N58W134', 'N57W134', 'N58W134', 'N56W130', 'N58W134', 'N55W131', 'N56W132', 'N58W134', 'N55W132', 'N57W135', 'N58W135', 'N58W134', 'N56W133', 'N58W134', 'N58W134', 'N57W134', 'N59W139', 'N56W133', 'N58W134', 'N58W134', 'N58W134', 'N56W130', 'N58W134', 'N55W131', 'N57W135', 'N55W133', 'N56W132', 'N58W134', 'N55W133', 'N56W132', 'N55W132', 'N56W131', 'N58W134', 'N56W132', 'N56W135', 'N58W134', 'N58W134', 'N58W134', 'N58W135', 'N58W134', 'N58W134', 'N55W131', 'N58W134', 'N57W135', 'N55W132', 'N58W134', 'N56W133', 'N55W131', 'N57W135', 'N56W132', 'N58W134', 'N55W133', 'N55W131', 'N56W135', 'N58W134', 'N58W135', 'N58W134', 'N58W134', 'N58W134', 'N56W132', 'N55W132', 'N58W134', 'N58W134', 'N56W132', 'N58W134', 'N56W132', 'N56W132', 'N56W132', 'N57W135', 'N58W135', 'N58W135', 'N56W132', 'N58W134', 'N58W135', 'N58W134', 'N56W133', 'N58W134', 'N58W135', 'N58W134', 'N58W134', 'N56W132', 'N56W133', 'N58W135', 'N55W131', 'N56W132', 'N55W131', 'N58W134', 'N58W134', 'N58W134', 'N58W134', 'N58W135', 'N58W134', 'N58W135', 'N58W134', 'N55W131', 'N58W134', 'N56W132', 'N55W132', 'N58W134', 'N58W135', 'N56W133', 'N58W135', 'N55W133', 'N55W131', 'N57W135', 'N56W132', 'N56W133', 'N58W134', 'N58W134', 'N56W132', 'N55W132', 'N58W134', 'N58W134', 'N55W131', 'N58W134', 'N58W134', 'N58W134', 'N56W133', 'N55W132', 'N58W134', 'N58W135', 'N58W134', 'N58W135', 'N58W134', 'N58W134', 'N57W135', 'N57W134', 'N55W131', 'N55W132', 'N57W135', 'N57W135', 'N58W135', 'N55W133', 'N58W134', 'N58W134', 'N56W132', 'N58W134', 'N56W132', 'N56W132', 'N56W132', 'N55W131', 'N58W134', 'N58W134', 'N58W134', 'N56W132', 'N55W131', 'N58W134', 'N58W134', 'N58W134', 'N58W134', 'N56W132']
#tile_urls = build_srtm1_urls(missing_tiles)


download_tiles(tile_urls, output_dir)


data/srtm1_hgt_tiles\S89E100.hgt.gz
data/srtm1_hgt_tiles\S89E100.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W145.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W145.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N60W148.hgt.gz
data/srtm1_hgt_tiles\N60W149.hgt.gz
data/srtm1_hgt_tiles\N59W149.hgt.gz
data/srtm1_hgt_tiles\N60W149

In [58]:
warnings.simplefilter(action='ignore', category=FutureWarning)

tile_dir = 'data/srtm_hgt_tiles'

# read gz file
def read_hgt_tile(tile_path):
    with gzip.open(tile_path, 'rb') as f:
        data = np.frombuffer(f.read(), dtype='>i2')
    return data

def reshape_to_grid(data, size=3601):
    if len(data) != size * size:
        raise ValueError(f"Data length mismatch: expected {size * size}, got {len(data)}")
    return data.reshape((size, size))

tile_cache = {}

missing_tiles = []

def get_elevation(lat, lon, tile_dir):
    lat_deg = int(lat)
    lon_deg = int(lon)
    
    lat_str = f"{abs(lat_deg):02d}"
    lon_str = f"{abs(lon_deg):03d}"
    
    if lat < 0:
        lat_str = 'S' + lat_str
    else:
        lat_str = 'N' + lat_str
    
    if lon < 0:
        lon_str = 'W' + lon_str
    else:
        lon_str = 'E' + lon_str

    tile_filename = f"{lat_str}{lon_str}.hgt.gz"
    tile_path = os.path.join(tile_dir, tile_filename)

    if not os.path.exists(tile_path):
        missing_tiles.append(tile_path)
        print(f"Tile {tile_filename} not found.")
        return None

    if tile_filename in tile_cache:
        elevation_data = tile_cache[tile_filename]
    else:
        data = read_hgt_tile(tile_path)
        elevation_data = reshape_to_grid(data)
        tile_cache[tile_filename] = elevation_data

    lat_idx = int((lat - lat_deg) * 3601)
    lon_idx = int((lon - lon_deg) * 3601)

    try:
        return elevation_data[lat_idx, lon_idx]
    except IndexError:
        return None

def enrich_with_elevation(fire_data, tile_dir):
    fire_data = fire_data.to_crs(epsg=4326)

    fire_data['DISCOVERYD'] = pd.to_datetime(fire_data['DISCOVERYD'], errors='coerce')
    fire_data['DISCOVERYD'].fillna(pd.Timestamp('1900-01-01'), inplace=True)

    # filter the data between 1980 and 2020
    fire_data['year'] = fire_data['DISCOVERYD'].dt.year
    fire_data = fire_data[(fire_data['year'] >= 1980) & (fire_data['year'] <= 2020)]

    with open('data/elevations.csv', 'w') as f:
        f.write("OBJECTID,ELEVATION")
        # iterate through each fire record and get the elevation
        for row in tqdm(fire_data.itertuples(), total=len(fire_data), desc="Processing fires"):
            # extract the fire record's geometry
            point = row.geometry
            lat, lon = point.y, point.x
            
            elevation = get_elevation(lat, lon, tile_dir)
            if elevation:
                f.write(f"\n{row.OBJECTID},{elevation}")

weather = pd.read_csv('data/weather_data_per_fire.csv')
matching_objectids = weather['OBJECTID'].unique()
usfs = gpd.read_file('data/National_USFS_Fire_Occurrence_Point_FL/usfs_fire_occurrence_cleaned_1980_2020.shp')
usfs_filtered = usfs[usfs['OBJECTID'].isin(matching_objectids)].copy()

merged = gpd.GeoDataFrame(usfs_filtered, geometry='geometry', crs=usfs.crs)

enrich_with_elevation(merged, tile_dir)


49071
49071
49071


Processing fires: 100%|████████████████████████████████████████████████████████| 49066/49066 [00:31<00:00, 1550.94it/s]


In [36]:
data = ['data/srtm_hgt_tiles\\S89E100.hgt.gz', 'data/srtm_hgt_tiles\\S89E100.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W145.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W145.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W148.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N59W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N60W149.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N55W132.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N57W135.hgt.gz', 'data/srtm_hgt_tiles\\N55W132.hgt.gz', 'data/srtm_hgt_tiles\\N57W135.hgt.gz', 'data/srtm_hgt_tiles\\N57W134.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N55W133.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N55W133.hgt.gz', 'data/srtm_hgt_tiles\\N55W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N57W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W130.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N55W132.hgt.gz', 'data/srtm_hgt_tiles\\N57W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W133.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N57W134.hgt.gz', 'data/srtm_hgt_tiles\\N59W139.hgt.gz', 'data/srtm_hgt_tiles\\N56W133.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W130.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N57W135.hgt.gz', 'data/srtm_hgt_tiles\\N55W133.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N55W133.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N55W132.hgt.gz', 'data/srtm_hgt_tiles\\N56W131.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N56W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N57W135.hgt.gz', 'data/srtm_hgt_tiles\\N55W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W133.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N57W135.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N55W133.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N56W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N55W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N57W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W133.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N56W133.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N55W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N56W133.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N55W133.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N57W135.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N56W133.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N55W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W133.hgt.gz', 'data/srtm_hgt_tiles\\N55W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N57W135.hgt.gz', 'data/srtm_hgt_tiles\\N57W134.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N55W132.hgt.gz', 'data/srtm_hgt_tiles\\N57W135.hgt.gz', 'data/srtm_hgt_tiles\\N57W135.hgt.gz', 'data/srtm_hgt_tiles\\N58W135.hgt.gz', 'data/srtm_hgt_tiles\\N55W133.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz', 'data/srtm_hgt_tiles\\N55W131.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N58W134.hgt.gz', 'data/srtm_hgt_tiles\\N56W132.hgt.gz']
new_data = []
for i in data:
    new_data.append(i.split("data/srtm_hgt_tiles\\")[1].split(".hgt.gz")[0])
print(new_data)

['S89E100', 'S89E100', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W145', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W145', 'N60W149', 'N60W149', 'N60W149', 'N60W148', 'N60W149', 'N59W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N60W149', 'N58W135', 'N55W132', 'N56W132', 'N57W135', 'N55W132', 'N57W135', 'N57W134', 'N55W131', 'N55W133', 'N58W134', 'N55W133', 'N55W132', 'N58W134', 'N58W134', 'N56W132', 'N58W135', 'N58W135', 'N58W134', 'N56W132', 'N58W134', 'N58W134', 'N58W134', 'N58W134', 'N57W134', 'N58W134', 'N56W130', 'N58W134', 'N55W131', 'N56W132', 'N58W134', 'N55W132', 'N57W135', 'N58W135', 'N58W134', 'N56W133', 'N58W134', 'N58W134', 'N57W134', 'N59W139', 'N56W133', 'N58W134', 'N58W134', 'N58W134', 'N56W130', 'N58W134', 'N55W131', 'N57W135', 'N55W133', 'N56W132', 'N58W134', 'N55W133', 'N56W132', 'N55W132', 'N56W131'